# Using a trained network for predictions


In [2]:
import os
import time
import pickle

import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
from keras import metrics

from cars_utils import pickle_variable_to_path

Using TensorFlow backend.


## Image, directory, and batch setup

In [3]:
# Image dimensions:
IMG_WIDTH, IMG_HEIGHT = 299, 299

WORKING_DIR = os.getcwd()
BASE = os.path.join(WORKING_DIR, 'data', 'cars_dataset', 'keras')

SAVE_DIR = os.path.join(WORKING_DIR, 'data', 'notebooks', '6_predictions_with_trained_model')

### Settings for full training
TEST_DATA_DIR = os.path.join(BASE, 'test')
NB_CLASSES = 196
BATCH_SIZE = 1

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

## Image augmentation generators

In [4]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [5]:
test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=13)

Found 2428 images belonging to 196 classes.


In [22]:
lookup_dicto = {value: key for key, value in test_generator.class_indices.items()}
lookup_dicto

{0: 'AM-General_Hummer_SUV-2000',
 1: 'Acura-Integra_Type_R-2001',
 2: 'Acura-RL_Sedan-2012',
 3: 'Acura-TL_Sedan-2012',
 4: 'Acura-TL_Type_S-2008',
 5: 'Acura-TSX_Sedan-2012',
 6: 'Acura-ZDX_Hatchback-2012',
 7: 'Aston_Martin-Martin_V8_Vantage_Convertible-2012',
 8: 'Aston_Martin-Martin_V8_Vantage_Coupe-2012',
 9: 'Aston_Martin-Martin_Virage_Convertible-2012',
 10: 'Aston_Martin-Martin_Virage_Coupe-2012',
 11: 'Audi-100_Sedan-1994',
 12: 'Audi-100_Wagon-1994',
 13: 'Audi-A5_Coupe-2012',
 14: 'Audi-R8_Coupe-2012',
 15: 'Audi-RS_4_Convertible-2008',
 16: 'Audi-S4_Sedan-2007',
 17: 'Audi-S4_Sedan-2012',
 18: 'Audi-S5_Convertible-2012',
 19: 'Audi-S5_Coupe-2012',
 20: 'Audi-S6_Sedan-2011',
 21: 'Audi-TTS_Coupe-2012',
 22: 'Audi-TT_Hatchback-2011',
 23: 'Audi-TT_RS_Coupe-2012',
 24: 'Audi-V8_Sedan-1994',
 25: 'BMW-1_Series_Convertible-2012',
 26: 'BMW-1_Series_Coupe-2012',
 27: 'BMW-3_Series_Sedan-2012',
 28: 'BMW-3_Series_Wagon-2012',
 29: 'BMW-6_Series_Convertible-2007',
 30: 'BMW-Active

In [6]:
# Save label indices for later 
pickle_variable_to_path(test_generator.class_indices, 'class_indices', SAVE_DIR)


********** pickle file already exists **********



In [31]:
pickle_variable_to_path(lookup_dicto, 'lookup_dicto', SAVE_DIR)


********** pickle file already exists **********



## Model definition

In [7]:
from keras.applications import InceptionV3

conv_base = InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=INPUT_SHAPE
)

In [8]:
pred_layer_config = {
    'activation': 'softmax',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
                           'config': {
                               'distribution': 'uniform',
                               'mode': 'fan_avg',
                               'scale': 1.0,
                               'seed': 8}
                          },
    'kernel_regularizer': None,
    'name': 'predictions',
    'trainable': True,
    'units': NB_CLASSES,
    'use_bias': True}

model = Sequential()
model.add(conv_base)
model.add(Dense(**pred_layer_config))

In [9]:
WEIGHT_PATH = os.path.join(WORKING_DIR, 'data', 'InceptionV3', 'InceptionV3_13_10_20.h5')
model.load_weights(WEIGHT_PATH)

In [26]:
from keras.preprocessing.image import load_img, img_to_array

input_image = load_img(os.path.join(os.path.expanduser('~'), 'Dropbox','projects',
                                    'django','cars','media','uploads',
                                    'Dodge-Ram_Pickup_3500_Crew_Cab-2010.jpg'), 
                       target_size=(IMG_HEIGHT, IMG_WIDTH))
input_image = img_to_array(input_image)
# the images are scaled during training so they need to be scaled for predictions too
input_image = input_image/255.
input_image = np.expand_dims(input_image, axis=0)

In [27]:
prediction = model.predict(input_image)

In [28]:
top_3 = prediction[0].argsort()[-3:][::-1]
top_3

array([ 91, 195,  60])

In [29]:
for idx in top_3:
    print(lookup_dicto.get(idx))

Dodge-Durango_SUV-2012
smart-fortwo_Convertible-2012
Chevrolet-Express_Van-2007
